In [2]:
import pandas as pd
import os
os.chdir("/group-volume/binfeng/wsdm/stage_qft")
from utils import *

import numpy as np

def softmax(logits):
    exp_logits = np.exp(logits - np.max(logits))  # Subtract max for numerical stability
    return exp_logits / np.sum(exp_logits)

def pred(x):
    logits = np.array([x.logits_model_a, x.logits_model_b])
    probs = softmax(logits)
    
    prob_model_a = probs[0]
    prob_model_b = probs[1]
    
    if prob_model_a > 0.5:
        return "model_a"
    else:
        return "model_b"
def merge_logits(x):
    return [x.logits_model_a, x.logits_model_b]

## HFOPEN and LMSYS

In [2]:
hfopen1 = pd.read_parquet("/group-volume/binfeng/wsdm/stage_qft/plabels/qwen32b_plabel_hfopen.parquet")
hfopen2 = pd.read_parquet("/group-volume/binfeng/wsdm/stage_qft/plabels/qwencd32b_plabel_hfopen.parquet")
lmsys1 = pd.read_parquet("/group-volume/binfeng/wsdm/stage_qft/plabels/qwen32b_plabel_lmsys.parquet")
lmsys2 = pd.read_parquet("/group-volume/binfeng/wsdm/stage_qft/plabels/qwencd32b_plabel_lmsys.parquet")


In [3]:
hfopen_m = pd.read_parquet("/group-volume/binfeng/wsdm/stage_qft/plabels/qwen32b_plabel_hfopen.parquet")
lmsys_m = pd.read_parquet("/group-volume/binfeng/wsdm/stage_qft/plabels/qwen32b_plabel_lmsys.parquet")
hfopen_m["logits_model_a"] = 0.5 * hfopen1["logits_model_a"] + 0.5 * hfopen2["logits_model_a"]
lmsys_m["logits_model_a"] = 0.5 * lmsys1["logits_model_a"] + 0.5 * lmsys2["logits_model_a"]
hfopen_m["logits_model_b"] = 0.5 * hfopen1["logits_model_b"] + 0.5 * hfopen2["logits_model_b"]
lmsys_m["logits_model_b"] = 0.5 * lmsys1["logits_model_b"] + 0.5 * lmsys2["logits_model_b"]

In [4]:
hfopen_m["winner"] = hfopen1.apply(pred, axis=1)
lmsys_m["winner"] = lmsys1.apply(pred, axis=1)

## Soft full

In [6]:
# soft
hfopen = pd.read_parquet("/group-volume/binfeng/wsdm/stage_qft/plabels/qwen32b_plabel_hfopen.parquet")
lmsys = pd.read_parquet("/group-volume/binfeng/wsdm/stage_qft/plabels/qwen32b_plabel_lmsys.parquet")
hfopen["winner"] = hfopen_m["winner"]
lmsys["winner"] = lmsys_m["winner"]
soft_full = pd.concat([hfopen, lmsys], axis=0)

In [7]:
soft_full["logits_qwen32"] = soft_full.apply(merge_logits, axis=1)
soft_full = soft_full[['prompt', 'response_a', 'response_b', 'winner', 'logits_qwen32']]

In [8]:
soft_full.dropna(inplace=True)
soft_full.reset_index(drop=True, inplace=True)
soft_full.shape

(87757, 5)

In [9]:
soft_full.to_parquet("/group-volume/binfeng/wsdm/stage_qft/data/soft87k.parquet", index=False)

## soft2

In [10]:
# soft
hfopen = pd.read_parquet("/group-volume/binfeng/wsdm/stage_qft/plabels/qwencd32b_plabel_hfopen.parquet")
lmsys = pd.read_parquet("/group-volume/binfeng/wsdm/stage_qft/plabels/qwencd32b_plabel_lmsys.parquet")
hfopen["winner"] = hfopen_m["winner"]
lmsys["winner"] = lmsys_m["winner"]
soft_full = pd.concat([hfopen, lmsys], axis=0)

In [11]:
soft_full["logits_qwencd"] = soft_full.apply(merge_logits, axis=1)
soft_full = soft_full[['prompt', 'response_a', 'response_b', 'winner', 'logits_qwencd']]

In [12]:
soft_full.dropna(inplace=True)
soft_full = soft_full.reset_index(drop=True)
soft_full.shape

(87757, 5)

In [13]:
soft_old = pd.read_parquet("/group-volume/binfeng/wsdm/stage_qft/data/soft87k.parquet")

In [14]:
soft_full["logits_qwen32"]  = soft_old["logits_qwen32"]

In [15]:
soft_full.head(10)

,prompt,response_a,response_b,winner,logits_qwencd,logits_qwen32
0,Cuadro Comparativo entre el Sistema Nervioso y...,El Sistema Nervioso y el Sistema Endocrino son...,A continuación se presenta un cuadro comparati...,model_a,"[0.18798828125, -0.166015625]","[-1.125, -1.48828125]"
1,Está escribiendo el guión de un curso de vídeo...,"Claro, aquí tienes el objeto JSONcon la inform...","{\n ""course_name"": ""Esencia del Vino: Un Vi...",model_b,"[-1.1796875, 1.16015625]","[-2.515625, -0.12060546875]"
2,Please give an xml hierarchy representing the ...,"```xml\n<?xml version=""1.0"" encoding=""UTF-8""?>...",Here's an XML hierarchy representing the anato...,model_a,"[-0.123291015625, 0.14111328125]","[-1.11328125, -1.515625]"
3,Tu es un consultant. Tu dois faire une proposi...,Voici une proposition de tableau récapitulatif...,Voici la liste des plateformes de recherche ex...,model_b,"[-0.09765625, 0.119384765625]","[-1.4609375, -1.1953125]"
4,extraia do texto abaixo os pedidos e a decisão...,| Pedido | Causa raiz ...,| Pedido | Causa Raiz do Pedido | Decisão (de...,model_a,"[-0.09375, 0.0830078125]","[-1.234375, -1.39453125]"
5,Please list the different types of cardiac mur...,Certainly! Here's a table summarizing the diff...,Here’s a table summarizing the different types...,model_a,"[0.1123046875, -0.104248046875]","[-1.0546875, -1.5859375]"
6,Transform the following C# Unity code to use U...,To transform the provided C# Unity code into u...,```csharp\nusing UniRx;\nusing UnityEngine;\n\...,model_b,"[-0.119140625, 0.077392578125]","[-1.43359375, -1.19140625]"
7,crie um swagger de um petshot\n,Aqui está um exemplo de um Swagger (OpenAPI Sp...,Aqui está um Swagger YAML para um API que regi...,model_a,"[0.4365234375, -0.421875]","[-0.80859375, -1.80078125]"
8,Show me source code to open an Intuition windo...,```c\n#include <intuition.h>\n\nvoid OpenIntui...,"Sure, here's an example of source code writte...",model_a,"[0.68359375, -0.705078125]","[-0.63671875, -1.8984375]"
9,\n\nI am building an AI powered UI design tool...,```json\nexport const EcommerceProductDetail: ...,"Sure, I can help you with that! Here's an exa...",model_b,"[-0.0126953125, 0.0225830078125]","[-1.4375, -1.1953125]"


In [17]:
soft_full.to_parquet("/group-volume/binfeng/wsdm/stage_qft/data/soft87k.parquet", index=False)

## kaggle data

In [2]:
kaggle1 = pd.read_parquet("/group-volume/binfeng/wsdm/stage_qft/plabels/qwen32b_plabel_kaggle.parquet")
kaggle2 = pd.read_parquet("/group-volume/binfeng/wsdm/stage_qft/plabels/qwencd32b_plabel_kaggle.parquet")


In [3]:
tmp = pd.read_parquet("/group-volume/binfeng/wsdm/stage_qft/data/ft48k_calibrated.parquet")

In [7]:
kaggle1["logits_qwen32_cali"] = kaggle1.apply(merge_logits, axis=1)
kaggle1["logits_qwencd_cali"] = kaggle2.apply(merge_logits, axis=1)

In [13]:
kaggle1.to_parquet("/group-volume/binfeng/wsdm/stage_qft/data/ft48k_calibrated.parquet")

## Kaggle Data (Final Reinforce)

In [3]:
kaggle1 = pd.read_parquet("/group-volume/binfeng/wsdm/stage_qft/plabels/qwen32b_plabel_kaggle.parquet")
kaggle2 = pd.read_parquet("/group-volume/binfeng/wsdm/stage_qft/plabels/qwencd32b_plabel_kaggle.parquet")
kaggle3 = pd.read_parquet("/group-volume/binfeng/wsdm/stage_qft/plabels/qwen14b_tie_plabel_kaggle.parquet")

In [4]:
tmp = pd.read_parquet("/group-volume/binfeng/wsdm/stage_qft/data/ft48k_calibrated.parquet")

In [6]:
kaggle1["logits_qwen32_cali"] = kaggle1.apply(merge_logits, axis=1)
kaggle1["logits_qwencd_cali"] = kaggle2.apply(merge_logits, axis=1)
kaggle1["logits_qwen14_cali"] = kaggle3.apply(merge_logits, axis=1)
kaggle1

,prompt,response_a,response_b,winner,language,text,text_reverse,input_ids,input_ids_reverse,fold,logits_model_a,logits_model_b,logits_qwen32_cali,logits_qwencd_cali,logits_qwen14_cali
0,draw a unicorn using asciii,Sure! Here's a simple ASCII art representation...,Sure! Here’s a simple ASCII art representation...,model_a,English,<|User Prompt|>\ndraw a unicorn using asciii\n...,<|User Prompt|>\ndraw a unicorn using asciii\n...,"[27, 91, 1474, 59501, 91, 397, 7633, 264, 8183...","[27, 91, 1474, 59501, 91, 397, 7633, 264, 8183...",0,-0.298828,-2.351562,"[-0.298828125, -2.3515625]","[0.849609375, -0.875]","[0.250244140625, -2.96875]"
1,"Сделать декомпозицию процесса ""выполнить ремон...","## Декомпозиция процесса ""Выполнить ремонт"" (D...","## Декомпозиция процесса ""Выполнить ремонт"" (д...",model_b,Russian,<|User Prompt|>\nСделать декомпозицию процесса...,<|User Prompt|>\nСделать декомпозицию процесса...,"[27, 91, 1474, 59501, 91, 397, 19311, 57930, 1...","[27, 91, 1474, 59501, 91, 397, 19311, 57930, 1...",1,-2.757812,0.110840,"[-2.7578125, 0.11083984375]","[-1.34765625, 1.296875]","[-3.0, 0.2568359375]"
2,NOMBRE DE LA ACTIVIDAD: Análisis descriptivo t...,"En el proceso de liofilización de alimentos, p...",## Análisis termodinámico de la liofilización:...,model_a,Spanish,<|User Prompt|>\nNOMBRE DE LA ACTIVIDAD: Análi...,<|User Prompt|>\nNOMBRE DE LA ACTIVIDAD: Análi...,"[27, 91, 1474, 59501, 91, 397, 45, 62011, 3385...","[27, 91, 1474, 59501, 91, 397, 45, 62011, 3385...",2,-0.080078,-2.523438,"[-0.080078125, -2.5234375]","[1.13671875, -1.16796875]","[-0.0933837890625, -2.625]"
3,find the logic error in this code and explain ...,The logic error in the provided code is relate...,The code is well-structured and clean. However...,model_a,English,<|User Prompt|>\nfind the logic error in this ...,<|User Prompt|>\nfind the logic error in this ...,"[27, 91, 1474, 59501, 91, 397, 3903, 279, 1221...","[27, 91, 1474, 59501, 91, 397, 3903, 279, 1221...",3,-0.859375,-1.820312,"[-0.859375, -1.8203125]","[0.36328125, -0.3662109375]","[0.34912109375, -3.09375]"
4,"\nДобрый день, коллега. От Вас требуется завер...",```python\n \n \n \n \n \n \...,"В вашем коде есть несколько импортов, которые ...",model_b,Russian,"<|User Prompt|>\n\nДобрый день, коллега. От Ва...","<|User Prompt|>\n\nДобрый день, коллега. От Ва...","[27, 91, 1474, 59501, 91, 1339, 24110, 131014,...","[27, 91, 1474, 59501, 91, 1339, 24110, 131014,...",0,-1.765625,-0.835938,"[-1.765625, -0.8359375]","[-1.109375, 1.09765625]","[-2.125, -0.513671875]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48432,原名赵亮的王雁南女士的爸爸，因为涉及了什么事件，被软禁起来？,王雁南女士的原名是赵亮，她的父亲是赵紫阳。赵紫阳是中国改革开放时期的重要领导人之一，曾担任国...,王雁南女士的父亲赵亮因涉及“反右派”事件而被软禁。反右派运动是中国在1957年进行的一次政治...,model_a,Chinese,<|User Prompt|>\n原名赵亮的王雁南女士的爸爸，因为涉及了什么事件，被软禁起来...,<|User Prompt|>\n原名赵亮的王雁南女士的爸爸，因为涉及了什么事件，被软禁起来...,"[27, 91, 1474, 59501, 91, 397, 52129, 13072, 1...","[27, 91, 1474, 59501, 91, 397, 52129, 13072, 1...",0,0.140442,-2.828125,"[0.14044189453125, -2.828125]","[0.728515625, -0.7578125]","[-1.04296875, -1.55859375]"
48433,\n太興燒味會否結業的\n,太興烧味是否会结业，这个问题的答案会受到多种因素的影响，包括但不限于市场环境、经营状况、管理...,截至我最後的資訊更新日期（2023年10月），我沒有具體的資訊表明太興燒味是否已經結業。太興...,model_a,Chinese,<|User Prompt|>\n\n太興燒味會否結業的\n\n\n<|Response A...,<|User Prompt|>\n\n太興燒味會否結業的\n\n\n<|Response A...,"[27, 91, 1474, 59501, 91, 1339, 99222, 103061,...","[27, 91, 1474, 59501, 91, 1339, 99222, 103061,...",1,-1.265625,-1.351562,"[-1.265625, -1.3515625]","[-0.076171875, 0.075927734375]","[-3.0546875, 0.23876953125]"
48434,葡萄糖是从葡萄中提取的吗,不是的。葡萄糖的名字可能会让人误以为它是从葡萄中提取的，但实际上，葡萄糖是一种单糖，也被称为...,葡萄糖并不是专门从葡萄中提取的，尽管它的名字听起来像是与葡萄有直接的联系。实际上，葡萄糖是一...,model_b,Chinese,<|User Prompt|>\n葡萄糖是从葡萄中提取的吗\n\n<|Response A|...,<|User Prompt|>\n葡萄糖是从葡萄中提取的吗\n\n<|Response A|...,"[27, 91, 1474, 59501, 91, 397, 101580, 100443,...","[27, 91, 1474, 59501, 91, 397, 101580, 100443,...",2,-1.492188,-1.128906,"[-1.4921875, -1.12890625]","[-0.088134765625, 0.093994140625]","[-0.2001953125, -2.5390625]"
48435,請寫一首關於台灣的七言律詩,島

In [7]:
kaggle1.to_parquet("/group-volume/binfeng/wsdm/stage_qft/data/ft48k_calibrated.parquet")

In [6]:
# def mean_logits(x):
#     return [(x["logits_qwen32_cali"][0]+x["logits_qwencd_cali"][0])/2, (x["logits_qwen32_cali"][1]+x["logits_qwencd_cali"][1])/2]

# kaggle1["logits_qwen32_cali"] = kaggle1.apply(mean_logits, axis=1)
# kaggle1["logits_qwencd_cali"] = kaggle3.apply(merge_logits, axis=1)
# kaggle1.head()

,prompt,response_a,response_b,winner,language,text,text_reverse,input_ids,input_ids_reverse,fold,logits_model_a,logits_model_b,logits_qwen32_cali,logits_qwencd_cali
0,draw a unicorn using asciii,Sure! Here's a simple ASCII art representation...,Sure! Here’s a simple ASCII art representation...,model_a,English,<|User Prompt|>\ndraw a unicorn using asciii\n...,<|User Prompt|>\ndraw a unicorn using asciii\n...,"[27, 91, 1474, 59501, 91, 397, 7633, 264, 8183...","[27, 91, 1474, 59501, 91, 397, 7633, 264, 8183...",0,-0.298828,-2.351562,"[0.275390625, -1.61328125]","[0.250244140625, -2.96875]"
1,"Сделать декомпозицию процесса ""выполнить ремон...","## Декомпозиция процесса ""Выполнить ремонт"" (D...","## Декомпозиция процесса ""Выполнить ремонт"" (д...",model_b,Russian,<|User Prompt|>\nСделать декомпозицию процесса...,<|User Prompt|>\nСделать декомпозицию процесса...,"[27, 91, 1474, 59501, 91, 397, 19311, 57930, 1...","[27, 91, 1474, 59501, 91, 397, 19311, 57930, 1...",1,-2.757812,0.110840,"[-2.052734375, 0.703857421875]","[-3.0, 0.2568359375]"
2,NOMBRE DE LA ACTIVIDAD: Análisis descriptivo t...,"En el proceso de liofilización de alimentos, p...",## Análisis termodinámico de la liofilización:...,model_a,Spanish,<|User Prompt|>\nNOMBRE DE LA ACTIVIDAD: Análi...,<|User Prompt|>\nNOMBRE DE LA ACTIVIDAD: Análi...,"[27, 91, 1474, 59501, 91, 397, 45, 62011, 3385...","[27, 91, 1474, 59501, 91, 397, 45, 62011, 3385...",2,-0.080078,-2.523438,"[0.5283203125, -1.845703125]","[-0.0933837890625, -2.625]"
3,find the logic error in this code and explain ...,The logic error in the provided code is relate...,The code is well-structured and clean. However...,model_a,English,<|User Prompt|>\nfind the logic error in this ...,<|User Prompt|>\nfind the logic error in this ...,"[27, 91, 1474, 59501, 91, 397, 3903, 279, 1221...","[27, 91, 1474, 59501, 91, 397, 3903, 279, 1221...",3,-0.859375,-1.820312,"[-0.248046875, -1.09326171875]","[0.34912109375, -3.09375]"
4,"\nДобрый день, коллега. От Вас требуется завер...",```python\n \n \n \n \n \n \...,"В вашем коде есть несколько импортов, которые ...",model_b,Russian,"<|User Prompt|>\n\nДобрый день, коллега. От Ва...","<|User Prompt|>\n\nДобрый день, коллега. От Ва...","[27, 91, 1474, 59501, 91, 1339, 24110, 131014,...","[27, 91, 1474, 59501, 91, 1339, 24110, 131014,...",0,-1.765625,-0.835938,"[-1.4375, 0.130859375]","[-2.125, -0.513671875]"


In [7]:
df = kaggle3.copy()
df["pred"] = df.apply(lambda x: "model_a" if x["logits_model_a"] > x["logits_model_b"] else "model_b", axis=1)
df["correct"] = df["pred"] == df["winner"]
hard_df = df[df["correct"]==False].reset_index(drop=True)
hard_df.shape

(5539, 14)

In [9]:
kaggle1 = kaggle1[kaggle1["prompt"].isin(hard_df["prompt"].tolist())]
kaggle1.shape

(6572, 14)

In [10]:
kaggle1.to_parquet("/group-volume/binfeng/wsdm/stage_qft/data/ftre6k_calibrated.parquet")

## Clean outliers

In [26]:
ppt = pd.read_parquet("/group-volume/binfeng/wsdm/stage_distill/datap/ppt135k.parquet")
ft = pd.read_parquet("/group-volume/binfeng/wsdm/stage_distill/datap/ft48k.parquet")

In [40]:
def prob(x):
    logits = np.array(np.mean([x["logits_qwencd"], x["logits_qwen32"]], axis=0))
    probs = softmax(logits)
    prob_model_a = probs[0]
    # prob_model_b = probs[1]
    return prob_model_a

ppt["a_win"] = ppt.apply(prob, axis=1)
ft["a_win"] = ft.apply(prob, axis=1)

In [72]:
def isoutlier(x):
    if x["winner"]=="model_a" and x["a_win"] < 0.3:
        return True
    if x["winner"]=="model_b" and x["a_win"] > 0.7:
        return True
    return False

ppt["outlier"] = ppt.apply(isoutlier, axis=1)
ft["outlier"] = ft.apply(isoutlier, axis=1)

In [78]:
def calibrate(outlier, logits):
    l = logits.copy()
    if outlier:
        l[0], l[1] = 0.5, 0.5
        return l
    else:
        return l
    
ppt["logits_qwencd_cali"] = ppt.apply(lambda x: calibrate(x["outlier"], x["logits_qwencd"]), axis=1)
ppt["logits_qwen32_cali"] = ppt.apply(lambda x: calibrate(x["outlier"], x["logits_qwen32"]), axis=1)
ft["logits_qwencd_cali"] = ft.apply(lambda x: calibrate(x["outlier"], x["logits_qwencd"]), axis=1)
ft["logits_qwen32_cali"] = ft.apply(lambda x: calibrate(x["outlier"], x["logits_qwen32"]), axis=1)

In [83]:
ft.drop(['a_win', 'outlier'], axis=1, inplace=True)
ppt.drop(['a_win', 'outlier'], axis=1, inplace=True)

In [84]:
ft.shape

(48437, 10)

In [85]:
ppt.shape

(196794, 8)

In [87]:
ft.to_parquet("/group-volume/binfeng/wsdm/stage_final/data/ft48k_calibrated.parquet")
ppt.to_parquet("/group-volume/binfeng/wsdm/stage_final/data/ppt196k_calibrated.parquet")